In [1]:
import re
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
import time
import random
import json
import html5lib


In [33]:
urls = [
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=carpenter',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=driver',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=electrician',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=software+engineer',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=pharmacist',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=secretary',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=nurse',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=childminder',
]

fileNames = [
    'carpentor',
    'driver',
    'electrician',
    'software_engineer',
    'pharmacist',
    'secretary',
    'nurse',
    'childminder'
    ]

In [34]:
jobNumberFinder = 'govuk-heading-l'
jobTitleFinder = "govuk-heading-s govuk-!-margin-top-4 govuk-!-margin-bottom-2"
jobDetailDescriptionFinder = 'govuk-body govuk-!-margin-bottom-6'

In [35]:
fileNameIndex = 0
failedTopUrls = []
failedDetailUrls = []
pageNumber = 1
for url in urls:
    data = []
    dataIndex = 1
    pageNumber = 1
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html5lib")
        time.sleep(0.5)
        response.close()
        jobNumberText = soup.find("h1", class_=jobNumberFinder).get_text()
        jobNumber = re.sub(r'\D', '', jobNumberText) if jobNumberText is not None else 0
        jobNumber = '5000' if int(jobNumber) > 5000 else jobNumber
        print("Success - url:" + url + "jobNumber:" + jobNumber)
    except ConnectionError:
        print('Fail - url:' + url)
        failedTopUrls.append(url)
        time.sleep(0.5)
        continue
    while(True):
        if(pageNumber == 1):
            titles = []
            titleAreas = soup.find_all("h3", class_=jobTitleFinder)
            for titleArea in titleAreas:
                titles.append(titleArea.findChildren("a" , recursive=False))
        else:
            try:
                newUrl = url.replace('&p\=[0-9]', '&p=' + str(pageNumber))
                response = requests.get(newUrl)
                soup = BeautifulSoup(response.content, "html5lib")
                time.sleep(0.5)
                response.close()
                titles = []
                titleAreas = soup.find_all("h3", class_=jobTitleFinder)
                for titleArea in titleAreas:
                    titles.append(titleArea.findChildren("a" , recursive=False))
            except ConnectionError:
                print('Fail - url:' + newUrl)
                failedTopUrls.append(url)
                pageNumber = pageNumber + 1
                if(int(jobNumber) >= pageNumber * 50):
                    break
                time.sleep(0.5)
                continue
        for title in titles:
            try:
                jobDetailResponse = requests.get(title[0]['href'])
                jobDetailSoup = BeautifulSoup(jobDetailResponse.content, "html5lib")
                time.sleep(0.5)
                jobDetailResponse.close()
                jobdescription = None if jobDetailSoup.find("div", class_=jobDetailDescriptionFinder) is None else jobDetailSoup.find("div", class_=jobDetailDescriptionFinder).get_text()
                data.append({str(dataIndex) : {'title': title[0].get_text(), 'description':jobdescription}})
                dataIndex = dataIndex + 1
            except ConnectionError:
                print('Fail - detail url:' + url)
                failedDetailUrls.append(title[0]['href'])
                time.sleep(0.5)
                continue
            time.sleep(random.randint(2,7))
        pageNumber = pageNumber + 1
        if(pageNumber * 50 >= int(jobNumber)):
            break
        time.sleep(random.randint(2,7))
    
    # export data into json file
    with open('./data/en/' + fileNames[fileNameIndex] + '.json', mode='w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
        print("Success - dump " + fileNames[fileNameIndex] + '.json')
    fileNameIndex = fileNameIndex + 1

# export Top Url data into json file
with open('./data/en/failedTopUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedTopUrls, file, ensure_ascii=False, indent=2)
    print("Success - dump " + 'failedTopUrls.json')
    
# export detailed Url data into json file
with open('./data/en/failedDetailUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedDetailUrls, file, ensure_ascii=False, indent=2)
    print("Success - dump " + 'failedDetailUrls.json')
    

Success - url:https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=nursejobNumber:5000
Success - dump nurse.json
Success - url:https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=childminderjobNumber:63
Success - dump childminder.json
Success - dump failedTopUrls.json
Success - dump failedDetailUrls.json
